# Cache everything in the Redis!

In [1]:
from datetime import datetime
import configparser
import json
import redis
from PIL import Image
import io
import utils

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
import redis_db

In [4]:
len(redis_db.thumb.keys())

5

In [6]:
import redis_db

In [7]:
redis_db.exif.flushall?

In [8]:
redis_db.exif.flushdb?

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

In [8]:
def redis_cache(fcn):
    def wrapper(fname):
        if isinstance(fname, bytes):
            fname=fname.decode("UTF-8")
            
        
        return 
    return wrapper

In [13]:
import xxhash

In [ ]:
@redis_cache

In [17]:
def get_xxhash(fname):
    """ Get the xxhash of a given file."""
    
    if isinstance(fname, bytes):
        fname=fname.decode("UTF-8")
    
    hash64 = xxhash.xxh64()
    with open(str(fname), "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash64.update(chunk)
    return hash64.hexdigest()

In [22]:
fname="/projects/darknet/data/dog.jpg"

In [18]:
get_xxhash("/projects/darknet/data/dog.jpg")

'7a4ee943299372fb'

In [20]:
get_xxhash.__name__.split("_")[1]

'xxhash'

In [24]:
db = get_xxhash.__name__.split("_")[1]
redis_db.redis_databases[db].exists(fname)

True

In [ ]:
def get_exif(fname):
    """Get the exif of a given file."""
    
    if isinstance(fname, bytes):
        fname=fname.decode("UTF-8")
        
    with exiftool.ExifTool() as et:
        return et.get_metadata(str(fname))

In [ ]:
redis_databases

In [ ]:
import sys

In [ ]:
thumbnail_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["thumb"],
)
xxhash_cache = redis.StrictRedis(
    host=config["redis"]["host"],
    port=config["redis"]["port"],
    db=config["redis"]["xxhash"],
)

In [ ]:
def get_thumbnail(img_path, size = (255, 255)):
    if isinstance(img_path, bytes):
        img_path=img_path.decode("UTF-8")
    
    img = Image.open(img_path)
    img.thumbnail(size)
    with io.BytesIO() as buffer:
        img.save(buffer, format="jpeg")
        thumbnail = buffer.getvalue()
    return thumbnail

def cache_thumbnail_redis(img_path, size = (255, 255)):
    if isinstance(img_path, bytes):
        img_path=img_path.decode("UTF-8")
    
    xxhash = utils.cache_xxhash(img_path)
    
    if thumbnail_cache.exists(xxhash):
        # Do nothing, thumbnail cache already exists.
        return None

    thumbnail = get_thumbnail(img_path=img_path, size=size)
    return thumbnail_cache.set(xxhash, thumbnail)

In [ ]:
cache_thumbnail_redis(xxhash_cache.randomkey())